In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def load_and_preprocess_image(image_path, closing_kernel = np.array([[0,0,0,0,0],[0,1,1,1,0],[1,1,1,1,1],[0,1,1,1,0],[0,0,1,0,0]],np.uint8)):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
   
    closing_kernel= closing_kernel
    processed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, closing_kernel, iterations=3)

    dilation_kernel = np.ones((2, 2), np.uint8)
    processed_image = cv2.dilate(processed_image, dilation_kernel, iterations=1)

    return image, processed_image


def draw_bounding_boxes(image, bounding_boxes, color=(0, 255, 0), thickness=2):

    output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    for (x, y, w, h) in bounding_boxes:
        cv2.rectangle(output_image, (x, y), (x + w, y + h), color, thickness)
    
    return output_image

def display_image(image, title="Optimized Character Segmentation"):
 
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.title(title)
    plt.show()



In [2]:
def split_large_boxes(processed_image, stats, original_image, width_threshold=1.26, height_threshold=1.3):
  
    new_boxes = [] 
    stats_len = len(stats)
    all_widths = [stats[i][2] for i in range(0,stats_len)]
   # print(all_widths)
    all_heights = [stats[i][3] for i in range(0,stats_len)]
    all_area = [stats[i][4] for i in range(0,stats_len)]
    avg_width = np.mean(all_widths) if all_widths else 20
    #print(avg_width)
    avg_height = np.mean(all_heights) if all_heights else 30
    #print(avg_height)
    avg_area = np.mean(all_area) if all_area else 100
    #print(avg_area)
    for x, y, w, h, area in stats:
        aspect_ratio_width = w / h
        aspect_ratio_height = h / w
       
        if aspect_ratio_width > width_threshold and w > 1.26*avg_width and area > 1.55*avg_area: 
            #print(area) 
            refined_boxes = refine_extreme_aspect_boxes(original_image, x,y,w,h, avg_height,avg_width, vertical = False)
            #print(refined_boxes)
            for x,y,w,h in refined_boxes:
                new_boxes.append((x,y,w,h))
        elif aspect_ratio_height > height_threshold and h > 1.26* avg_height and area > 1.6* avg_area:  
            refined_boxes = refine_extreme_aspect_boxes(original_image, x,y,w,h, avg_height,avg_width, vertical = True)
            #print(refined_boxes)
            for x,y,w,h in refined_boxes:
                new_boxes.append((x, y, w, h))
        else:
            new_boxes.append((x, y, w, h))
      
   # print(new_boxes)

    return new_boxes

def refine_extreme_aspect_boxes(image, x,y,w,h, avg_height, avg_width, closing_kernel_size=(5,3), tolerance_ratio=0.5, vertical = False, iteration = 2):
    closing_kernel = np.array([[0,1,0],[0,1,0],[0,1,1],[1,1,1],[1,1,1],[0,1,0],[0,1,0]],np.uint8)
    connectivity = 8
    if vertical:
        closing_kernel_size = (2,3)
        closing_kernel = np.ones(closing_kernel_size, np.uint8)
        iteration = 2
        connectivity = 8
    refined_boxes = []
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
            # Extract the region of interest (ROI) from the original image
    boxed_region = image[y:y+h, x:x+w]
    #boxed_img = draw_bounding_boxes(image,[(x,y,w,h)])
    #display_image(boxed_img)

            # Apply threshold and smaller morphological closing to this ROI
          
    boxed_region_closed = cv2.morphologyEx(boxed_region, cv2.MORPH_CLOSE, closing_kernel, iterations=iteration)
    
   

            # Detect connected components in the ROI
    num, _, stats, _ = cv2.connectedComponentsWithStats(boxed_region_closed, connectivity)

    best_local_box = None
    for i in range(1, num):
        lx, ly, lw, lh, larea = stats[i]
        if lw < 0.5*avg_width or lh < 0.5*avg_height:
            continue
                # Check if the box is close to the average dimensions
        if (abs(lw - avg_width) / avg_width < tolerance_ratio and
            abs(lh - avg_height) / avg_height < tolerance_ratio):
            best_local_box = (x + lx, y + ly, lw, lh)
        if best_local_box:
            #print(f"best_local_box:{best_local_box}")
            refined_boxes.append(best_local_box)
 

    if not refined_boxes:
        refined_boxes.append((x, y, w, h))
        
    #boxed_img = draw_bounding_boxes(image,refined_boxes)
    #display_image(boxed_img, title=f"{refined_boxes}")
     
    return refined_boxes

def detect_large_characters(processed_image, original_image, min_area=50, max_area=5000, dynamic_factor=0.45, padding_ratio=0.1):
   
    num_labels, labels, stats, centboxed_regionds = cv2.connectedComponentsWithStats(processed_image, connectivity=8)
    w_thresh = 10
    h_thresh = 10
    img_h, img_w = processed_image.shape
    large_char_stats= []
    for x, y, w, h, area in stats:
        if (w > w_thresh and h >= h_thresh):
            large_char_stats.append((x, y, w, h, area))
    large_char_num = len(large_char_stats)
    all_widths = [large_char_stats[i][2] for i in range(1, large_char_num)]
    all_heights = [large_char_stats[i][3] for i in range(1, large_char_num)]
    all_areas = [large_char_stats[i][4] for i in range(1, large_char_num)]
    avg_area = np.mean(all_areas) if all_widths else 600
    avg_width = np.mean(all_widths) if all_widths else 20
    avg_height = np.mean(all_heights) if all_heights else 30

    min_width = max(15, int(avg_width * dynamic_factor))
    max_width = int(avg_width * 2.5)
    min_height = max(15, int(avg_height * dynamic_factor))
    max_height = int(avg_height * 2.8)    
    max_area = int(avg_area * 3.2)


    bounding_boxes = []
    for i in range(1, large_char_num):  
        x, y, w, h, area = large_char_stats[i]

        if min_area < area < max_area and min_width < w < max_width and min_height < h < max_height:
            bounding_boxes.append((x, y, w, h, area))

    refined_boxes = split_large_boxes(processed_image, bounding_boxes, original_image)

    final_boxes = []
    for x, y, w, h in refined_boxes:
        padding_w = int(w * padding_ratio)
        padding_h = int(h * padding_ratio)

        x_new = max(0, x - padding_w)
        y_new = max(0, y - padding_h)
        w_new = min(img_w - x_new, w + 2 * padding_w)
        h_new = min(img_h - y_new, h + 2 * padding_h)

        final_boxes.append((x_new, y_new, w_new, h_new))
    
    return final_boxes


In [3]:
import os
def save_image(image, input_path, output_folder):
    """saves image"""
    filename = os.path.basename(input_path) 
    output_path = os.path.join(output_folder, filename) 
    os.makedirs(output_folder, exist_ok=True) 
    #save without compression
    cv2.imwrite(output_path, image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    print(f"saved:{output_path}")

import os
import cv2
import numpy as np
from tqdm import tqdm

def save_characters_from_boxes(image, boxes, image_path, output_root='segmented_chars_img', min_size=(5, 5)):
    """
    Save each character from bounding boxes into a structured output folder.
    Skips very small boxes (less than min_size).
    """
    # Prepare paths
    image_dir = os.path.dirname(image_path)           # e.g., output/RC04844
    subfolder = os.path.basename(image_dir)            # e.g., RC04844
    image_filename = os.path.basename(image_path)      # e.g., page_1.png
    image_name = os.path.splitext(image_filename)[0]   # e.g., page_1

    output_folder = os.path.join(output_root, subfolder, image_name)
    os.makedirs(output_folder, exist_ok=True)

    # Sort boxes top to bottom, then left to right
    boxes_sorted = sorted(boxes, key=lambda b: (b[1], b[0]))

    count = 0
    for (x, y, w, h) in boxes_sorted:
        # Skip tiny noisy boxes
        if w < min_size[0] or h < min_size[1]:
            continue

        crop = image[y:y+h, x:x+w]
        count += 1
        save_path = os.path.join(output_folder, f"c{count}.png")
        cv2.imwrite(save_path, crop, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        print(f"saved:{save_path}")

def process_all_images(input_root_folder='output_sample', output_root_folder='seg_visual_img', char_output_root='segmented_chars_img'):
    """
    Processes all images under input_root_folder (your "output" folder).
    Saves segmented characters under char_output_root,
    and full images with drawn boxes under output_root_folder.
    """
    for root, _, files in os.walk(input_root_folder):

        relative_path = os.path.relpath(root, input_root_folder)
        output_folder = os.path.join(output_root_folder, relative_path)
        os.makedirs(output_folder, exist_ok=True)

        # Sort filenames naturally (page_1.png before page_10.png)
        png_files = sorted(
            [file for file in files if file.lower().endswith(".png")],
            key=lambda x: (int(x.split('_')[-1].split('.')[0]) if '_' in x else x)
        )

        if not png_files:
            print(f"No PNG images found in {root}")
            continue

        for file in tqdm(png_files, desc=f"Processing {relative_path}"):
            input_image_path = os.path.join(root, file)
            

                # Load and preprocess
            original_image, processed_image = load_and_preprocess_image(input_image_path)
            print(input_image_path)
                # Detect characters
            character_boxes = detect_large_characters(processed_image, original_image)

                # Draw bounding boxes
            output_image_segmented = draw_bounding_boxes(original_image, character_boxes)

                # Save the segmented full image (for visualization)
            save_image(output_image_segmented, input_image_path, output_folder)

                # Save each cropped character
            save_characters_from_boxes(original_image, character_boxes, input_image_path, output_root=char_output_root)



In [10]:
input_root = r"C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\img_sample"
output_root = r"C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\visual"
char_output = r"C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars"
process_all_images(input_root_folder=input_root,output_root_folder=output_root,char_output_root= char_output)



No PNG images found in C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\img_sample


Processing RC05216:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\img_sample\RC05216\page_1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\visual\RC05216\page_1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c2.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c3.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c4.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CU

Processing RC05216: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]


saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c542.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c543.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c544.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c545.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c546.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05216\page_1\c547.png
saved:C:\U

...

saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\visual\RC05277\page_1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05277\page_1\c1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05277\page_1\c2.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05277\page_1\c3.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05277\page_1\c4.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05277\page_1\c5.png
saved:C:\Users\xiong\Des

Processing RC05716:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\img_sample\RC05716\page_1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\visual\RC05716\page_1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c1.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c2.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c3.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c4.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CU

Processing RC05716: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c248.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c249.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c250.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c251.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c252.png
saved:C:\Users\xiong\Desktop\SCHOOL\ANU\CUR_2025s1\Project\japanese-handwriting-analysis\image_processing_jyx\image_data\seg_v2_output\chars\RC05716\page_1\c253.png
saved:C:\U

In [12]:
import cv2
import os

def interactive_crop_with_move_and_resize(image, save_folder, base_filename, start_idx=1):
    clone = image.copy()
    display_img = image.copy()
    rois = []  # Save all rectangles (x, y, w, h)
    selected_idx = -1  # Index of the currently selected rectangle
    drawing = False    # Whether a new rectangle is being drawn
    moving = False     # Whether an existing rectangle is being moved
    resizing = False   # Whether an existing rectangle is being resized
    ix, iy = -1, -1    # Starting coordinates
    move_offset = (0, 0)
    resize_dir = None  # Resize direction: 'horizontal' or 'vertical'

    def point_inside_rect(x, y, rect):
        rx, ry, rw, rh = rect
        return rx <= x <= rx+rw and ry <= y <= ry+rh

    def point_near_edge(x, y, rect, edge_thickness=10):
        rx, ry, rw, rh = rect
        near_right = abs((rx+rw) - x) <= edge_thickness and ry <= y <= ry+rh
        near_bottom = abs((ry+rh) - y) <= edge_thickness and rx <= x <= rx+rw
        if near_right:
            return 'horizontal'
        if near_bottom:
            return 'vertical'
        return None

    def draw_all():
        temp = clone.copy()
        for idx, (x, y, w, h) in enumerate(rois):
            color = (0, 255, 0) if idx != selected_idx else (0, 0, 255)
            cv2.rectangle(temp, (x, y), (x+w, y+h), color, 2)
        return temp

    def mouse_callback(event, x, y, flags, param):
        nonlocal ix, iy, drawing, moving, resizing, selected_idx, move_offset, resize_dir, display_img

        if event == cv2.EVENT_LBUTTONDOWN:
            # Check if an existing rectangle was clicked
            selected_idx = -1
            for idx, rect in enumerate(rois):
                if point_inside_rect(x, y, rect):
                    selected_idx = idx
                    move_offset = (x - rect[0], y - rect[1])
                    mode = point_near_edge(x, y, rect)
                    if mode == 'horizontal' or mode == 'vertical':
                        resizing = True
                        resize_dir = mode
                    else:
                        moving = True
                    break

            if selected_idx == -1:
                # Not clicked on existing rectangle, start new
                drawing = True
                ix, iy = x, y

        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                display_img = draw_all()
                cv2.rectangle(display_img, (ix, iy), (x, y), (255, 0, 0), 2)
            elif moving and selected_idx != -1:
                dx, dy, w, h = rois[selected_idx]
                new_x = x - move_offset[0]
                new_y = y - move_offset[1]
                rois[selected_idx] = (new_x, new_y, w, h)
                display_img = draw_all()
            elif resizing and selected_idx != -1:
                dx, dy, w, h = rois[selected_idx]
                if resize_dir == 'horizontal':
                    new_w = max(1, x - dx)
                    rois[selected_idx] = (dx, dy, new_w, h)
                elif resize_dir == 'vertical':
                    new_h = max(1, y - dy)
                    rois[selected_idx] = (dx, dy, w, new_h)
                display_img = draw_all()

        elif event == cv2.EVENT_LBUTTONUP:
            if drawing:
                w, h = abs(x - ix), abs(y - iy)
                x1, y1 = min(ix, x), min(iy, y)
                if w > 5 and h > 5:  # Filter out rectangles that are too small
                    rois.append((x1, y1, w, h))
                drawing = False
                display_img = draw_all()
            moving = False
            resizing = False

    # Initialize window
    cv2.namedWindow("Select character regions - Press Enter to save, ESC to exit", cv2.WINDOW_NORMAL)
    cv2.setMouseCallback("Select character regions - Press Enter to save, ESC to exit", mouse_callback)

    while True:
        cv2.imshow("Select character regions - Press Enter to save, ESC to exit", display_img)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            print("❌ Exit without saving")
            break
        elif key == 13:  # Enter
            if not rois:
                print("⚠️ No regions selected")
                break
            os.makedirs(save_folder, exist_ok=True)
            for idx, (x, y, w, h) in enumerate(rois, start=start_idx):
                cropped = clone[y:y+h, x:x+w]
                save_path = os.path.join(save_folder, f"c{idx}.png")
                cv2.imwrite(save_path, cropped)
                print(f"✅ Saved: {save_path}")
            break

    cv2.destroyAllWindows()
    return start_idx + len(rois)


In [19]:
# Process all images: automatic detection + manual selection
def process_all_images(input_root_folder='output_sample', output_root_folder='seg_visual_img', char_output_root='segmented_chars_img'):
    for root, _, files in os.walk(input_root_folder):
        relative_path = os.path.relpath(root, input_root_folder)
        output_folder = os.path.join(output_root_folder, relative_path)
        os.makedirs(output_folder, exist_ok=True)

        png_files = sorted(
            [file for file in files if file.lower().endswith(".png")],
            key=lambda x: (int(x.split('_')[-1].split('.')[0]) if '_' in x else x)
        )

        if not png_files:
            print(f"No PNG images found in {root}")
            continue

        for file in tqdm(png_files, desc=f"Processing {relative_path}"):
            input_image_path = os.path.join(root, file)

            # 1. Load and preprocess the image
            original_image, processed_image = load_and_preprocess_image(input_image_path)
            print(f"▶️ Processing: {input_image_path}")

            # 2. Automatically detect character boxes
            character_boxes = detect_large_characters(processed_image, original_image)

            # 3. Draw the detected character boxes
            output_image_segmented = draw_bounding_boxes(original_image, character_boxes)

            # 4. Save the full image with drawn boxes
            save_image(output_image_segmented, input_image_path, output_folder)

            # 5. Save each auto-detected character as a cropped image
            save_characters_from_boxes(original_image, character_boxes, input_image_path, output_root=char_output_root)

            # 6. [New] Manually supplement with additional character crops
            # --- Prepare subfolder to save manual crops ---
            image_dir = os.path.dirname(input_image_path)
            subfolder = os.path.basename(image_dir)
            image_filename = os.path.basename(input_image_path)
            image_name = os.path.splitext(image_filename)[0]
            manual_save_folder = os.path.join(char_output_root, subfolder, image_name)
            os.makedirs(manual_save_folder, exist_ok=True)

            # --- Load the already processed image for manual selection ---
            processed_img_for_manual = cv2.imread(os.path.join(output_folder, image_filename))

            # --- Get the number of already auto-saved crops to continue indexing ---
            existing_files = os.listdir(manual_save_folder)
            auto_saved_count = len([f for f in existing_files if f.startswith('c') and f.endswith('.png')])
            start_idx = auto_saved_count + 1

            # --- ✨ Launch manual cropping window for refinement ---
            start_idx = interactive_crop_with_move_and_resize(
                processed_img_for_manual, manual_save_folder, image_name, start_idx=start_idx
            )


In [20]:
import os

def get_incremental_folders_in_range(input_root, start_folder, end_folder):
    """
    Get all subfolders between start_folder and end_folder, based on actual sorted folder names.
    """
    all_folders = [f for f in os.listdir(input_root) if os.path.isdir(os.path.join(input_root, f)) and f.startswith("RC")]
    sorted_folders = sorted(all_folders)

    if start_folder not in sorted_folders or end_folder not in sorted_folders:
        raise ValueError("Start or end folder not found in directory list.")

    start_idx = sorted_folders.index(start_folder)
    end_idx = sorted_folders.index(end_folder)

    return sorted_folders[start_idx:end_idx + 1]

def run_segmentation_pipeline_by_actual_range(input_root, output_root, char_output_root,
                                              start_folder="RC04876", end_folder="RC04890"):
    """
    Process all subfolders between start_folder and end_folder (inclusive),
    based on actual folder names in the directory, regardless of continuity.
    """
    selected_folders = get_incremental_folders_in_range(input_root, start_folder, end_folder)

    for folder_name in selected_folders:
        input_subfolder = os.path.join(input_root, folder_name)
        if not os.path.exists(input_subfolder):
            print(f"⚠️ Subfolder does not exist, skipping: {input_subfolder}")
            continue

        print(f"\n📂 Processing subfolder: {folder_name}")
        process_all_images(
            input_root_folder=input_subfolder,
            output_root_folder=os.path.join(output_root, folder_name),
            char_output_root=char_output_root
        )



In [21]:
run_segmentation_pipeline_by_actual_range(
    input_root="output_2",
    output_root="seg_visual_img",
    char_output_root="segmented_chars_img",
    start_folder="RC04857",
    end_folder="RC04867"
)




📂 Processing subfolder: RC04844


Processing .:   0%|          | 0/4 [00:00<?, ?it/s]

▶️ Processing: output_2\RC04844\page_1.png
saved:seg_visual_img\RC04844\.\page_1.png
saved:segmented_chars_img\RC04844\page_1\c1.png
saved:segmented_chars_img\RC04844\page_1\c2.png
saved:segmented_chars_img\RC04844\page_1\c3.png
saved:segmented_chars_img\RC04844\page_1\c4.png
saved:segmented_chars_img\RC04844\page_1\c5.png
saved:segmented_chars_img\RC04844\page_1\c6.png
saved:segmented_chars_img\RC04844\page_1\c7.png
saved:segmented_chars_img\RC04844\page_1\c8.png
saved:segmented_chars_img\RC04844\page_1\c9.png
saved:segmented_chars_img\RC04844\page_1\c10.png
saved:segmented_chars_img\RC04844\page_1\c11.png
saved:segmented_chars_img\RC04844\page_1\c12.png
saved:segmented_chars_img\RC04844\page_1\c13.png
saved:segmented_chars_img\RC04844\page_1\c14.png
saved:segmented_chars_img\RC04844\page_1\c15.png
saved:segmented_chars_img\RC04844\page_1\c16.png
saved:segmented_chars_img\RC04844\page_1\c17.png
saved:segmented_chars_img\RC04844\page_1\c18.png
saved:segmented_chars_img\RC04844\page_1\c

Processing .:  25%|██▌       | 1/4 [00:04<00:12,  4.27s/it]

⚠️ 没有框选任何区域
▶️ Processing: output_2\RC04844\page_2.png
saved:seg_visual_img\RC04844\.\page_2.png
saved:segmented_chars_img\RC04844\page_2\c1.png
saved:segmented_chars_img\RC04844\page_2\c2.png
saved:segmented_chars_img\RC04844\page_2\c3.png
saved:segmented_chars_img\RC04844\page_2\c4.png
saved:segmented_chars_img\RC04844\page_2\c5.png
saved:segmented_chars_img\RC04844\page_2\c6.png
saved:segmented_chars_img\RC04844\page_2\c7.png
saved:segmented_chars_img\RC04844\page_2\c8.png
saved:segmented_chars_img\RC04844\page_2\c9.png
saved:segmented_chars_img\RC04844\page_2\c10.png
saved:segmented_chars_img\RC04844\page_2\c11.png
saved:segmented_chars_img\RC04844\page_2\c12.png
saved:segmented_chars_img\RC04844\page_2\c13.png
saved:segmented_chars_img\RC04844\page_2\c14.png
saved:segmented_chars_img\RC04844\page_2\c15.png
saved:segmented_chars_img\RC04844\page_2\c16.png
saved:segmented_chars_img\RC04844\page_2\c17.png
saved:segmented_chars_img\RC04844\page_2\c18.png
saved:segmented_chars_img\RC04

...

⚠️ 没有框选任何区域

📂 Processing subfolder: RC04845


Processing .:   0%|          | 0/1 [00:00<?, ?it/s]

▶️ Processing: output_2\RC04845\page_1.png
saved:seg_visual_img\RC04845\.\page_1.png
saved:segmented_chars_img\RC04845\page_1\c1.png
saved:segmented_chars_img\RC04845\page_1\c2.png
saved:segmented_chars_img\RC04845\page_1\c3.png
saved:segmented_chars_img\RC04845\page_1\c4.png
saved:segmented_chars_img\RC04845\page_1\c5.png
saved:segmented_chars_img\RC04845\page_1\c6.png
saved:segmented_chars_img\RC04845\page_1\c7.png
saved:segmented_chars_img\RC04845\page_1\c8.png
saved:segmented_chars_img\RC04845\page_1\c9.png
saved:segmented_chars_img\RC04845\page_1\c10.png
saved:segmented_chars_img\RC04845\page_1\c11.png
saved:segmented_chars_img\RC04845\page_1\c12.png
saved:segmented_chars_img\RC04845\page_1\c13.png
saved:segmented_chars_img\RC04845\page_1\c14.png
saved:segmented_chars_img\RC04845\page_1\c15.png
saved:segmented_chars_img\RC04845\page_1\c16.png
saved:segmented_chars_img\RC04845\page_1\c17.png
saved:segmented_chars_img\RC04845\page_1\c18.png
saved:segmented_chars_img\RC04845\page_1\c

Processing .: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]

⚠️ 没有框选任何区域
